<a href="https://colab.research.google.com/github/ShaswataJash/PytorchVisionModelComparision/blob/main/Pytorch_vision_model_comparision.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Download required python dependency (torchvision and torchinfo)

In [1]:
!pip install torchvision==0.14.0

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
!pip install torchinfo==1.7.1

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [3]:
import torchvision
from torchvision.models import *
from torchinfo import summary
from tqdm import tqdm
import pandas as pd
import pprint

In [4]:
def my_import(name): #ref: https://stackoverflow.com/questions/547829/how-to-dynamically-load-a-python-class
    components = name.split('.')
    mod = __import__(components[0])
    for comp in components[1:]:
        mod = getattr(mod, comp)
    return mod

# Vision model comparision

The main logic is to compare models according to 
* 5% accuracy
* Training time (for that number of multiplications and additions involved in the model is the representative metric)
* memory required in CPU or GPU for a batch. For batch, we assume the size as (2, 3, [recommended crop width for the model], [recommended crop height for the model]) 

The higher the accuracy the better is the model. The lower the training time required, the better is the model. The lower the memory required for the batch, the better is the model.

In [5]:
classification_models = list_models(module=torchvision.models)
pprint.pprint(classification_models)

['alexnet',
 'convnext_base',
 'convnext_large',
 'convnext_small',
 'convnext_tiny',
 'densenet121',
 'densenet161',
 'densenet169',
 'densenet201',
 'efficientnet_b0',
 'efficientnet_b1',
 'efficientnet_b2',
 'efficientnet_b3',
 'efficientnet_b4',
 'efficientnet_b5',
 'efficientnet_b6',
 'efficientnet_b7',
 'efficientnet_v2_l',
 'efficientnet_v2_m',
 'efficientnet_v2_s',
 'googlenet',
 'inception_v3',
 'maxvit_t',
 'mnasnet0_5',
 'mnasnet0_75',
 'mnasnet1_0',
 'mnasnet1_3',
 'mobilenet_v2',
 'mobilenet_v3_large',
 'mobilenet_v3_small',
 'regnet_x_16gf',
 'regnet_x_1_6gf',
 'regnet_x_32gf',
 'regnet_x_3_2gf',
 'regnet_x_400mf',
 'regnet_x_800mf',
 'regnet_x_8gf',
 'regnet_y_128gf',
 'regnet_y_16gf',
 'regnet_y_1_6gf',
 'regnet_y_32gf',
 'regnet_y_3_2gf',
 'regnet_y_400mf',
 'regnet_y_800mf',
 'regnet_y_8gf',
 'resnet101',
 'resnet152',
 'resnet18',
 'resnet34',
 'resnet50',
 'resnext101_32x8d',
 'resnext101_64x4d',
 'resnext50_32x4d',
 'shufflenet_v2_x0_5',
 'shufflenet_v2_x1_0',
 'sh

In [6]:
square_image_size = 479 #@param {type:"integer"}
batch_size = 2 #@param {type:"integer"}
model_name = []
accuracy_1 = []
accuracy_5 = []
best_weight_name = []
crop_for_best_accu = []
total_mul_add = []
total_gpu_mem_mb_reqd = []
param_count_calculated = []
param_count_from_meta = []

for c in tqdm(classification_models):
   
    model_wt = get_model_weights(c)
    print(c, type(model_wt), model_wt.__name__)

    crop_size_corr_to_best_acc = None
    weight_name_corr_to_best_acc = None
    accu1_corr_to_best_acc = None
    param_count_corr_to_best_acc = None
    best_acc_5 = 0.0
    for wts in model_wt:
        print(wts.name, wts.value)
        print(wts.name, wts.value.transforms)
        print(wts.value.transforms.keywords['crop_size'])
        print(wts.value.meta['_metrics']['ImageNet-1K']['acc@1'])
        print(wts.value.meta['_metrics']['ImageNet-1K']['acc@5'])
        if wts.value.meta['_metrics']['ImageNet-1K']['acc@5'] > best_acc_5:
            best_acc_5 = wts.value.meta['_metrics']['ImageNet-1K']['acc@5']
            crop_size_corr_to_best_acc = wts.value.transforms.keywords['crop_size']
            weight_name_corr_to_best_acc = model_wt.__name__ + '.' + wts.name
            accu1_corr_to_best_acc = wts.value.meta['_metrics']['ImageNet-1K']['acc@1']
            param_count_corr_to_best_acc = wts.value.meta['num_params']

    model_name.append(c)
    accuracy_5.append(best_acc_5)
    accuracy_1.append(accu1_corr_to_best_acc)
    best_weight_name.append(weight_name_corr_to_best_acc)
    crop_for_best_accu.append(crop_size_corr_to_best_acc)
    param_count_from_meta.append(param_count_corr_to_best_acc)

    wt_enum = my_import('torchvision.models.'+weight_name_corr_to_best_acc)
    m = get_model(c, weights=wt_enum)
    print(c, type(m), type(wt_enum), wt_enum.name)
    if square_image_size > 0:
        square_image_size = crop_size_corr_to_best_acc
    try:
        sum_obj = summary(m, 
            input_size=(batch_size, 3,square_image_size,square_image_size), #batch size = 2 (keep it more than 1 or else error)
            col_width=15,
            depth=8,
            mode='train',
            col_names=["input_size", "output_size", "num_params", "kernel_size"])
        gpu_bytes_required = sum_obj.to_megabytes(sum_obj.total_input + sum_obj.total_output_bytes + sum_obj.total_param_bytes)
        print(sum_obj.total_mult_adds,  gpu_bytes_required)
        
        total_mul_add.append(sum_obj.total_mult_adds)
        total_gpu_mem_mb_reqd.append(gpu_bytes_required)
        param_count_calculated.append(sum_obj.total_params)
        assert param_count_corr_to_best_acc == sum_obj.total_params

        del sum_obj
    except:
        print('failure observed for model', c)

        total_mul_add.append(np.nan)
        total_gpu_mem_mb_reqd.append(np.nan)
        param_count_calculated.append(np.nan)

    print("=========================")
    del model_wt
    del m

model_comparision_data = {'model': model_name, 'weight': best_weight_name, 
                          'accuracy_5': accuracy_5, 'accuracy_1': accuracy_1, 
                          'total_mul_add': total_mul_add, 'param_meta':param_count_from_meta,
                          'param_calcu': param_count_calculated,
                          'gpu_mem_reqd': total_gpu_mem_mb_reqd, 'crop_size': crop_for_best_accu}
comp_df = pd.DataFrame.from_dict(model_comparision_data)

  0%|          | 0/80 [00:00<?, ?it/s]

alexnet <class 'torchvision._utils.StrEnumMeta'> AlexNet_Weights
IMAGENET1K_V1 Weights(url='https://download.pytorch.org/models/alexnet-owt-7be5be79.pth', transforms=functools.partial(<class 'torchvision.transforms._presets.ImageClassification'>, crop_size=224), meta={'num_params': 61100840, 'min_size': (63, 63), 'categories': ['tench', 'goldfish', 'great white shark', 'tiger shark', 'hammerhead', 'electric ray', 'stingray', 'cock', 'hen', 'ostrich', 'brambling', 'goldfinch', 'house finch', 'junco', 'indigo bunting', 'robin', 'bulbul', 'jay', 'magpie', 'chickadee', 'water ouzel', 'kite', 'bald eagle', 'vulture', 'great grey owl', 'European fire salamander', 'common newt', 'eft', 'spotted salamander', 'axolotl', 'bullfrog', 'tree frog', 'tailed frog', 'loggerhead', 'leatherback turtle', 'mud turtle', 'terrapin', 'box turtle', 'banded gecko', 'common iguana', 'American chameleon', 'whiptail', 'agama', 'frilled lizard', 'alligator lizard', 'Gila monster', 'green lizard', 'African chameleo

Downloading: "https://download.pytorch.org/models/alexnet-owt-7be5be79.pth" to /root/.cache/torch/hub/checkpoints/alexnet-owt-7be5be79.pth


  0%|          | 0.00/233M [00:00<?, ?B/s]

alexnet <class 'torchvision.models.alexnet.AlexNet'> <enum 'AlexNet_Weights'> IMAGENET1K_V1


...

Downloading: "https://download.pytorch.org/models/wide_resnet50_2-9ba9bcbe.pth" to /root/.cache/torch/hub/checkpoints/wide_resnet50_2-9ba9bcbe.pth


  0%|          | 0.00/263M [00:00<?, ?B/s]

wide_resnet50_2 <class 'torchvision.models.resnet.ResNet'> <enum 'Wide_ResNet50_2_Weights'> IMAGENET1K_V2


100%|██████████| 80/80 [13:11<00:00,  9.89s/it]

22796180688 737.569616


In [7]:
pd.set_option('display.max_rows', comp_df.shape[0]+1)
comp_df_filtered = comp_df.dropna()
comp_df_filtered.sort_values(by=['accuracy_5'], ascending=False)

,model,weight,accuracy_5,accuracy_1,total_mul_add,param_meta,param_calcu,gpu_mem_reqd,crop_size
75,vit_h_14,ViT_H_14_Weights.IMAGENET1K_SWAG_E2E_V1,98.694,88.552,2906398160,633470440,633470440,8034.799024,518
37,regnet_y_128gf,RegNet_Y_128GF_Weights.IMAGENET1K_SWAG_E2E_V1,98.682,88.228,749141004284,644812894,644812894,9307.779976,384
76,vit_l_16,ViT_L_16_Weights.IMAGENET1K_SWAG_E2E_V1,98.512,88.064,2017859536,305174504,305174504,3674.652240,512
40,regnet_y_32gf,RegNet_Y_32GF_Weights.IMAGENET1K_SWAG_E2E_V1,98.362,86.838,189653294500,145046770,145046770,3427.166232,384
38,regnet_y_16gf,RegNet_Y_16GF_Weights.IMAGENET1K_SWAG_E2E_V1,98.054,86.012,93470087480,83590140,83590140,2502.860256,384
17,efficientnet_v2_l,EfficientNet_V2_L_Weights.IMAGENET1K_V1,97.788,85.808,112160664272,118515272,118515272,5533.269968,480
73,vit_b_16,ViT_B_16_Weights.IMAGENET1K_SWAG_E2E_V1,97.650,85.304,795315152,86859496,86859496,845.566544,384
18,efficientnet_v2_m,EfficientNet_V2_M_Weights.IMAGENET1K_V1,97.156,85.112,49165642472,54139356,54139356,3095.815392,480
2,convnext_large,ConvNeXt_Large_Weights.IMAGENET1K_V1,96.976,84.414,2527644752,197767336,197767336,1620.712016,224
15,efficientnet_b6,EfficientNet_B6_Weights.IMAGENET1K_V1,96.916,84.008,38135743136,43040704,43040704,5534.584880,528


In [8]:
comp_df_filtered = comp_df_filtered.astype({'param_meta':'int', 'total_mul_add':'int', 'param_calcu': 'int'})
comp_df_filtered

,model,weight,accuracy_5,accuracy_1,total_mul_add,param_meta,param_calcu,gpu_mem_reqd,crop_size
0,alexnet,AlexNet_Weights.IMAGENET1K_V1,79.066,56.522,1429365328,61100840,61100840,253.514576,224
1,convnext_base,ConvNeXt_Base_Weights.IMAGENET1K_V1,96.870,84.062,1293060816,88591464,88591464,907.867728,224
2,convnext_large,ConvNeXt_Large_Weights.IMAGENET1K_V1,96.976,84.414,2527644752,197767336,197767336,1620.712016,224
3,convnext_small,ConvNeXt_Small_Weights.IMAGENET1K_V1,96.650,83.616,822782480,50223688,50223688,616.326224,224
4,convnext_tiny,ConvNeXt_Tiny_Weights.IMAGENET1K_V1,96.146,82.520,644743184,28589128,28589128,378.083408,224
5,densenet121,DenseNet121_Weights.IMAGENET1K_V1,91.972,74.434,5668492624,7978856,7978856,394.202192,224
6,densenet161,DenseNet161_Weights.IMAGENET1K_V1,93.560,77.138,15456256016,28681000,28681000,763.590992,224
7,densenet169,DenseNet169_Weights.IMAGENET1K_V1,92.806,75.600,6720005456,14149480,14149480,478.995536,224
8,densenet201,DenseNet201_Weights.IMAGENET1K_V1,93.370,76.896,8583191888,20013928,20013928,602.203216,224
9,efficientnet_b0,EfficientNet_B0_Weights.IMAGENET1K_V1,93.532,77.692,771734216,5288548,5288548,238.130176,224


We have to normalize the values for accuracy_5, total_mul_add and gpu_mem_reqd to effectively compare different models in the same scale.

In [9]:
df_z_scaled = comp_df_filtered.copy()
df_z_scaled['z_accuracy_5'] = (df_z_scaled['accuracy_5'] - df_z_scaled['accuracy_5'].mean()) / df_z_scaled['accuracy_5'].std()
df_z_scaled['z_total_mul_add'] = (df_z_scaled['total_mul_add'] - df_z_scaled['total_mul_add'].mean()) / df_z_scaled['total_mul_add'].std()    
df_z_scaled['z_param_meta'] = (df_z_scaled['param_meta'] - df_z_scaled['param_meta'].mean()) / df_z_scaled['param_meta'].std()
df_z_scaled['z_gpu_mem_reqd'] = (df_z_scaled['gpu_mem_reqd'] - df_z_scaled['gpu_mem_reqd'].mean()) / df_z_scaled['gpu_mem_reqd'].std()
df_z_scaled

,model,weight,accuracy_5,accuracy_1,total_mul_add,param_meta,param_calcu,gpu_mem_reqd,crop_size,z_accuracy_5,z_total_mul_add,z_param_meta,z_gpu_mem_reqd
0,alexnet,AlexNet_Weights.IMAGENET1K_V1,79.066,56.522,1429365328,61100840,61100840,253.514576,224,-3.484616,-0.271885,-0.079255,-0.485453
1,convnext_base,ConvNeXt_Base_Weights.IMAGENET1K_V1,96.870,84.062,1293060816,88591464,88591464,907.867728,224,0.781288,-0.273453,0.168388,-0.138138
2,convnext_large,ConvNeXt_Large_Weights.IMAGENET1K_V1,96.976,84.414,2527644752,197767336,197767336,1620.712016,224,0.806686,-0.259251,1.151873,0.240223
3,convnext_small,ConvNeXt_Small_Weights.IMAGENET1K_V1,96.650,83.616,822782480,50223688,50223688,616.326224,224,0.728575,-0.278863,-0.177239,-0.292881
4,convnext_tiny,ConvNeXt_Tiny_Weights.IMAGENET1K_V1,96.146,82.520,644743184,28589128,28589128,378.083408,224,0.607815,-0.280911,-0.372129,-0.419335
5,densenet121,DenseNet121_Weights.IMAGENET1K_V1,91.972,74.434,5668492624,7978856,7978856,394.202192,224,-0.392291,-0.223120,-0.557791,-0.410780
6,densenet161,DenseNet161_Weights.IMAGENET1K_V1,93.560,77.138,15456256016,28681000,28681000,763.590992,224,-0.011800,-0.110526,-0.371301,-0.214717
7,densenet169,DenseNet169_Weights.IMAGENET1K_V1,92.806,75.600,6720005456,14149480,14149480,478.995536,224,-0.192462,-0.211024,-0.502205,-0.365773
8,densenet201,DenseNet201_Weights.IMAGENET1K_V1,93.370,76.896,8583191888,20013928,20013928,602.203216,224,-0.057325,-0.189591,-0.449376,-0.300378
9,efficientnet_b0,EfficientNet_B0_Weights.IMAGENET1K_V1,93.532,77.692,771734216,5288548,5288548,238.130176,224,-0.018509,-0.279450,-0.582026,-0.493619


We can configure relative weights for each of the metric that will contribute in overall score against each model. 

In [10]:
accuracy_weight = 0.6 #@param {type:"number"}
param_meta_weight = 0.3 #@param {type:"number"}
gpu_mem_weight = 0.05 #@param {type:"number"}
total_mul_add_weight = 0.05 #@param {type:"number"}
df_z_scaled['score'] = (accuracy_weight * df_z_scaled['z_accuracy_5']) \
     - (param_meta_weight * df_z_scaled['z_param_meta']) \
     - (gpu_mem_weight * df_z_scaled['z_gpu_mem_reqd']) \
     - (total_mul_add_weight * df_z_scaled['z_total_mul_add'])

In [11]:
df_z_scaled.sort_values(by=['score'], ascending=False)

,model,weight,accuracy_5,accuracy_1,total_mul_add,param_meta,param_calcu,gpu_mem_reqd,crop_size,z_accuracy_5,z_total_mul_add,z_param_meta,z_gpu_mem_reqd,score
19,efficientnet_v2_s,EfficientNet_V2_S_Weights.IMAGENET1K_V1,96.878,84.228,16731614432,21458488,21458488,1233.325840,384,0.783204,-0.095855,-0.436363,0.034607,0.603894
22,maxvit_t,MaxVit_T_Weights.IMAGENET1K_V1,96.722,83.700,4027148096,30919624,30919624,904.518704,224,0.745826,-0.242001,-0.351135,-0.139916,0.571932
13,efficientnet_b4,EfficientNet_B4_Weights.IMAGENET1K_V1,96.594,83.384,8787852528,19341616,19341616,1664.000496,380,0.715157,-0.187236,-0.455433,0.263200,0.561926
73,vit_b_16,ViT_B_16_Weights.IMAGENET1K_SWAG_E2E_V1,97.650,85.304,795315152,86859496,86859496,845.566544,384,0.968179,-0.279179,0.152786,-0.171206,0.557590
63,swin_v2_s,Swin_V2_S_Weights.IMAGENET1K_V1,96.816,83.712,106338512,49737442,49737442,525.893792,256,0.768349,-0.287105,-0.181619,-0.340881,0.546894
12,efficientnet_b3,EfficientNet_B3_Weights.IMAGENET1K_V1,96.054,82.008,3654499536,12233232,12233232,813.357168,300,0.585771,-0.246288,-0.519467,-0.188302,0.529032
38,regnet_y_16gf,RegNet_Y_16GF_Weights.IMAGENET1K_SWAG_E2E_V1,98.054,86.012,93470087480,83590140,83590140,2502.860256,384,1.064978,0.786911,0.123335,0.708447,0.527219
3,convnext_small,ConvNeXt_Small_Weights.IMAGENET1K_V1,96.650,83.616,822782480,50223688,50223688,616.326224,224,0.728575,-0.278863,-0.177239,-0.292881,0.518904
64,swin_v2_t,Swin_V2_T_Weights.IMAGENET1K_V1,96.132,82.072,77851856,28351570,28351570,325.297184,256,0.604460,-0.287432,-0.374269,-0.447353,0.511696
4,convnext_tiny,ConvNeXt_Tiny_Weights.IMAGENET1K_V1,96.146,82.520,644743184,28589128,28589128,378.083408,224,0.607815,-0.280911,-0.372129,-0.419335,0.511340
